# Recurrent Neural Networks

* so far our vanilla NNs expected a fixed-length input and predicted a fixed-length output (e.g., animacy)

* the exciting idea behind Recurrent Neural Network (RNN) is they allows us to work on **sequences** of input, output, or both.
* for example, if you want to predict the next word, you would like to know what word was before

### Dependence on previous step

* RNNs are called **recurrent** because they predict the next output being dependent on the previous output (i.e., like having a *memory* of what has been seen so far)


<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg" alt="illustration from WildML">
A recurrent neural network and the unfolding in time of the computation involved in its forward computation (LeCun et al., 2015).

### RNNs:  **sequences** of input, output, or both

Karpathy's illustration of RNNs:
<img src="http://benjaminbolte.com/resources/attention_rnn/karpathy_rnn.jpeg">

* From left to right: (1) Vanilla mode of processing without RNN, from fixed-sized input to fixed-sized output (e.g. image classification). (2) Sequence output (e.g. image captioning takes an image and outputs a sentence of words). (3) Sequence input (e.g. sentiment analysis where a given sentence is classified as expressing positive or negative sentiment). (4) Sequence input and sequence output (e.g. Machine Translation: an RNN reads a sentence in English and then outputs a sentence in French). (5) Synced sequence input and output (e.g. video classification where we wish to label each frame of the video). Notice that in every case are no pre-specified constraints on the lengths sequences because the recurrent transformation (green) is fixed and can be applied as many times as we like.*

## References

* [Karpathy's blog on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [WildMl blog](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)
    